<a href="https://colab.research.google.com/github/monarkparekh/MAE-598__Design-Optimization/blob/Project-1/MAE_598__Design_Optimization_Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>